In [ ]:
from torch.autograd import Variable
import numpy as np
import tensorflow as tf
import time, math, glob
import scipy.io as sio
import torch
import torch.nn as nn
from math import sqrt
import argparse, os
import torch
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.data as data
import h5py

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import copy

import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from PIL.ImageOps import colorize

import h5py as h5py
import numpy as np
import tensorflow as tf
from sklearn import metrics
import torchvision.models as models
from PIL import Image
import imageio
import matplotlib.pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive"
os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['Colab Notebooks',
 'content',
 'drive',
 'train',
 'validation',
 'Cifar10(half-half)_128_lr=0.05_epochs=50.h5',
 'Cifar10(half-half)_128_lr=0.05_epochs=50.h5（副本）',
 'checkpoint',
 'Cifar10_Aug',
 'merged_Cifar10(origin)_128_lr=0.002_epochs=100.h5',
 '0',
 '5',
 'tensorboard',
 'Cifar10_split',
 'TIP Moire Pattern Removal',
 'data']

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(128),
        transforms.RandomRotation(20),
        transforms.ColorJitter(),
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(128),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = '/content/drive/MyDrive/Cifar10-VDSR(256)_split'
data_dir = '/content/drive/MyDrive/Cifar10_split'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=8, drop_last=True)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, num_channels=3, upscale_factor=8, d=64, s=12, m=4):
        super(Net, self).__init__()

        self.first_part = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=d, kernel_size=5, stride=1, padding=2),
                                        nn.PReLU())

        self.layers = []
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))
        for _ in range(m):
            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1))
        self.layers.append(nn.PReLU())
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))

        self.mid_part = torch.nn.Sequential(*self.layers)

        # Deconvolution
        self.last_part = nn.ConvTranspose2d(in_channels=d, out_channels=num_channels, kernel_size=9, stride=upscale_factor, padding=3, output_padding=1)

    def forward(self, x):
        out = self.first_part(x)
        out = self.mid_part(out)
        out = self.last_part(out)
        return out

    def weight_init(self, mean=0.0, std=0.02):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0.0, 0.0001)
                if m.bias is not None:
                    m.bias.data.zero_()

def conv_layer(chann_in, chann_out, k_size, p_size):
    layer = nn.Sequential(
        nn.Conv2d(chann_in, chann_out, kernel_size=k_size, padding=p_size),
        nn.BatchNorm2d(chann_out),
        nn.ReLU()
    )
    return layer

def vgg_fc_layer(size_in, size_out):
    layer = nn.Sequential(
        nn.Linear(size_in, size_out),
        nn.BatchNorm1d(size_out),
        nn.ReLU()
    )
    return layer

def vgg_conv_block(in_list, out_list, k_list, p_list, pooling_k, pooling_s):

    layers = [conv_layer(in_list[i], out_list[i], k_list[i], p_list[i]) for i in range(len(in_list))]
    layers += [nn.MaxPool2d(kernel_size=pooling_k, stride=pooling_s)]
    return nn.Sequential(*layers)

class SDCNN(nn.Module):
    def __init__(self, n_classes=10):
        super(SDCNN, self).__init__()

        # Conv blocks (BatchNorm + ReLU activation added in each block)
        self.layer1 = vgg_conv_block([3,32], [32,32], [3,3], [1,1], 2, 2)
        self.layer2 = vgg_conv_block([32,64], [64,64], [3,3], [1,1], 2, 2)

        # FC layers
        self.layer3 = vgg_fc_layer(63*63*64, 512)  # 4096->smaller

        # Final layer
        self.layer4 = nn.Linear(512, 10)

    def forward(self, x):
        # 从这里开始
        out = self.layer1(x)
        features = self.layer2(out)
        out = features.view(out.size(0), -1)
        out = self.layer3(out)
        out = self.layer4(out)

        return out

In [ ]:
import torch
torch.cuda.set_device(0)

import os
os.environ['CUDA_ENABLE_DEVICES'] = '0'

In [ ]:
# Training settings
parser = argparse.ArgumentParser(description="PyTorch VDSR")
parser.add_argument("--nEpochs", type=int, default=50, help="Number of epochs to train for")
parser.add_argument("--lr", type=float, default=0.1, help="Learning Rate. Default=0.1")
parser.add_argument("--step", type=int, default=10, help="Sets the learning rate to the initial LR decayed by momentum every n epochs, Default: n=10")
parser.add_argument("--cuda", action="store_true", help="Use cuda?")
parser.add_argument("--resume", default="",type=str, help="Path to checkpoint (default: none)")
parser.add_argument("--start-epoch", default=1, type=int, help="Manual epoch number (useful on restarts)")
parser.add_argument("--clip", type=float, default=0.4, help="Clipping Gradients. Default=0.4")
parser.add_argument("--threads", type=int, default=1, help="Number of threads for data loader to use, Default: 1")
parser.add_argument("--momentum", default=0.9, type=float, help="Momentum, Default: 0.9")
parser.add_argument("--weight-decay", "--wd", default=1e-4, type=float, help="Weight decay, Default: 1e-4")
parser.add_argument('--pretrained_SR', default='/content/drive/MyDrive/checkpoint/model_FSRCNN_CNET_epoch_0.pth', type=str, help='path to pretrained model (default: none)')
parser.add_argument('--pretrained_TL', default='/content/drive/MyDrive/checkpoint/TransferLearning/model_CNET_epoch_0.pth', type=str, help='path to pretrained model (default: none)')
parser.add_argument("--gpus", default="0", type=str, help="gpu ids (default: 0)")
parser.add_argument("--pretrained_SR_num", default=0, type=int, help="numbers of epochs that have been trained")
parser.add_argument("--pretrained_TL_num", default=0, type=int, help="numbers of epochs that have been trained")
parser.add_argument("--SRtrain", default=False, help="if train the super resolution network")
parser.add_argument("--TLtrain", default=True, help="if train the trainsfer learning network")
parser.add_argument("--SR_used", default=False, help="if use the SR method thought the pipeline")

def main():
    global opt
    opt = parser.parse_args(args=[])
    print(opt)

    cuda = opt.cuda
    if cuda:
        print("=> use gpu id: '{}'".format(opt.gpus))
        os.environ["CUDA_VISIBLE_DEVICES"] = opt.gpus
        if not torch.cuda.is_available():
                raise Exception("No GPU found or Wrong gpu id, please run without --cuda")

    opt.seed = random.randint(1, 10000)
    print("Random Seed: ", opt.seed)
    torch.manual_seed(opt.seed)
    if cuda:
        torch.cuda.manual_seed(opt.seed)

    cudnn.benchmark = True

    print("===> Loading datasets")
    training_data_loader = dataloaders

    print("===> Building model")
    SRmodel = Net()
    CNET = SDCNN()
    criterion = nn.CrossEntropyLoss()

    print("===> Setting GPU")
    SRmodel = SRmodel.cuda()
    criterion = criterion.cuda()
    CNET = CNET.cuda()

    # optionally resume from a checkpoint
    if opt.resume:
        if os.path.isfile(opt.resume):
            print("=> loading checkpoint '{}'".format(opt.resume))
            checkpoint = torch.load(opt.resume)
            opt.start_epoch = checkpoint["epoch"] + 1
            SRmodel.load_state_dict(checkpoint["model"].state_dict())
        else:
            print("=> no checkpoint found at '{}'".format(opt.resume))

    # optionally copy weights from a checkpoint
    if opt.pretrained_SR:
        if os.path.isfile(opt.pretrained_SR):
            print("=> loading SR model '{}'".format(opt.pretrained_SR))
            weights = torch.load(opt.pretrained_SR)
            SRmodel.load_state_dict(weights['model'].state_dict())
        else:
            print("=> no model found at '{}'".format(opt.pretrained_SR))  
    
    if opt.pretrained_TL:
        if os.path.isfile(opt.pretrained_TL):
            print("=> loading TL model '{}'".format(opt.pretrained_TL))
            weights = torch.load(opt.pretrained_TL)
            #CNET.load_state_dict(weights)
            CNET.load_state_dict(weights['model'].state_dict())
        else:
            print("=> no model found at '{}'".format(opt.pretrained_TL)) 

    print("===> Setting Optimizer")
    optimizer_SR = optim.SGD(SRmodel.parameters(), lr=opt.lr, momentum=opt.momentum, weight_decay=opt.weight_decay)
    #optimizer_TL = optim.SGD(vgg16.fc.parameters(), lr=opt.lr, momentum=opt.momentum, weight_decay=opt.weight_decay)
    #optimizer_TL = optim.SGD(vgg16.parameters(), lr=opt.lr, momentum=opt.momentum, weight_decay=opt.weight_decay)
    optimizer_TL = optim.SGD(CNET.parameters(), lr=opt.lr, momentum=opt.momentum, weight_decay=opt.weight_decay)


    print("===> Training")
    for epoch in range(opt.start_epoch, opt.nEpochs + 1):
        train(dataloaders, optimizer_SR, optimizer_TL, SRmodel, CNET, criterion, epoch)
        if opt.SR_used:
            save_SR_checkpoint(SRmodel, epoch)
        if opt.TLtrain:
            save_TL_checkpoint(CNET, epoch)

def adjust_learning_rate(optimizer_SR, optimizer_TL, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = opt.lr * (0.1 ** (epoch // opt.step))
    return lr

def train(training_data_loader, optimizer_SR, optimizer_TL, SRmodel, CNET, criterion, epoch):
    lr = adjust_learning_rate(optimizer_SR, optimizer_TL, epoch-1)

    for param_group in optimizer_SR.param_groups:
        param_group["lr"] = lr

    for param_group in optimizer_TL.param_groups:
        param_group["lr"] = lr

    print("Epoch = {}, lr = {}".format(epoch, optimizer_SR.param_groups[0]["lr"]))

    #total = 0  
    #count = 0
    #false = 0

    SRmodel.train()

    total = 0

    for iteration, batch in enumerate(training_data_loader['train'], 1):

      images, labels = Variable(batch[0]), Variable(batch[1], requires_grad=False)
      images = images.cuda()
      labels = labels.cuda()

      if opt.SR_used:
        out_images = SRmodel(images)
        output = CNET(out_images)
      else:
        output = CNET(images)
      
      predicted = output
      #total += labels.size(0)
      #false += (torch.max(predicted.cpu(),1) != labels).sum()
      #count = count+1

      loss = criterion(predicted,labels)
      if opt.SRtrain:
        optimizer_SR.zero_grad()
      if opt.TLtrain:
        optimizer_TL.zero_grad()
      
      loss.backward() 
      nn.utils.clip_grad_norm_(SRmodel.parameters(),opt.clip)
      nn.utils.clip_grad_norm_(CNET.parameters(),opt.clip)

      if opt.SRtrain:
        optimizer_SR.step()
      if opt.TLtrain:
        optimizer_TL.step()

      total += loss.item()

      if iteration%100 == 0:
          print("===> Average Loss: {:.10f}".format(total/iteration))

    if opt.SR_used:
        # Get a batch of training data
        out_images, classes = out_images.cpu(), labels.cpu()
        inputs = images.cpu()

        # Make a grid from batch
        out_images = torchvision.utils.make_grid(out_images)
        inp = torchvision.utils.make_grid(inputs)

        imshow(out_images, title=[class_names[x] for x in classes])
        imshow(inp, title=[class_names[x] for x in classes])

def imshow(inp, title=None):
      """Imshow for Tensor."""
      inp = inp.numpy().transpose((1, 2, 0))
      mean = np.array([0.485, 0.456, 0.406])
      std = np.array([0.229, 0.224, 0.225])
      inp = std * inp + mean
      inp = np.clip(inp, 0, 1)
      plt.imshow(inp)
      if title is not None:
         plt.title(title)
      plt.pause(0.001)  # pause a bit so that plots are updated

def save_SR_checkpoint(SRmodel, epoch):
    model_out_path = "/content/drive/MyDrive/checkpoint/" + "model_FSRCNN_CNET_*8_epoch_{}.pth".format(epoch+opt.pretrained_SR_num)
    state = {"epoch": epoch ,"model": SRmodel}
    if not os.path.exists("/content/drive/MyDrive/checkpoint/"):
        os.makedirs("/content/drive/MyDrive/checkpoint/")

    torch.save(state, model_out_path)

    print("Super resolution network checkpoint saved to {}".format(model_out_path))

def save_TL_checkpoint(model, epoch):
    model_out_path = "/content/drive/MyDrive/checkpoint/TransferLearning/" + "model_FSRCNN_CNET_*8_epoch_{}.pth".format(epoch+opt.pretrained_TL_num)
    state = {"epoch": epoch ,"model": model}
    if not os.path.exists("/content/drive/MyDrive/checkpoint/TransferLearning/"):
        os.makedirs("/content/drive/MyDrive/checkpoint/TransferLearning/")

    torch.save(state, model_out_path)

    print("Transfer learning checkpoint saved to {}".format(model_out_path))

if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.